In [ ]:
import pandas as pd
import numpy as np
import random
import difflib
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

# Load dataset
df = pd.read_csv("C:/Users/muthu/Downloads/deficiency_recommendation_dataset.csv")# update path if needed

# Define columns
input_col = 'Deficiency'
output_cols = ['Recommended Food 1', 'Recommended Food 2', 'Notes']

# Encode outputs into a single action string
df['Action'] = df[output_cols].agg('|'.join, axis=1)
action_encoder = LabelEncoder()
df['Action Encoded'] = action_encoder.fit_transform(df['Action'])

# NLP: Map all deficiencies to lowercase for easy comparison
unique_deficiencies = df[input_col].unique()
normalized_deficiencies = [defi.lower() for defi in unique_deficiencies]

# Q-Learning Setup
states = df[input_col].values
actions = df['Action Encoded'].values
Q = defaultdict(lambda: np.zeros(len(action_encoder.classes_)))

# Hyperparameters
alpha = 0.5  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate
epochs = 10

# Training loop
for _ in range(epochs):
    for i in range(len(states)):
        state = states[i]
        action = actions[i]
        reward = 1  # Static reward

        if random.uniform(0, 1) < epsilon:
            next_action = random.randint(0, len(action_encoder.classes_) - 1)
        else:
            next_action = np.argmax(Q[state])

        Q[state][action] += alpha * (reward + gamma * np.max(Q[state]) - Q[state][action])

# Suggestion function with NLP-based correction
def suggest_food_for_deficiency():
    user_input = input("Enter deficiency (e.g., Vitamin C, Iron): ").strip().lower()
    
    # NLP-based close match
    closest_match = difflib.get_close_matches(user_input, normalized_deficiencies, n=1, cutoff=0.6)
    
    if not closest_match:
        print("Deficiency not recognized. Please try again.")
        return
    
    matched_def = unique_deficiencies[normalized_deficiencies.index(closest_match[0])]
    print(f"\nInterpreted input as: {matched_def}")

    if matched_def not in Q:
        print("No matching state found in Q-table. Suggestion may not be optimal.")
        action_id = random.randint(0, len(action_encoder.classes_) - 1)
    else:
        action_id = np.argmax(Q[matched_def])

    action_decoded = action_encoder.inverse_transform([action_id])[0]
    food1, food2, note = action_decoded.split('|')

    print("\n✅ Recommended Foods & Notes:")
    print(f"Recommended Food 1: {food1}")
    print(f"Recommended Food 2: {food2}")
    print(f"Note: {note}")

# Run
suggest_food_for_deficiency()


Interpreted input as: Vitamin D

✅ Recommended Foods & Notes:
Recommended Food 1: Cod liver oil
Recommended Food 2: Egg yolks
Note: Consume Cod liver oil and Egg yolks regularly to address Vitamin D deficiency.
